In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import nltk
nltk.download("punkt")
nltk.download("wordnet")
nltk.download("stopwords")

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Dropout, Embedding
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
df = pd.read_csv('tweets.csv')
df

,id,keyword,location,text,target
0,0,ablaze,NaN,"Communal violence in Bhainsa, Telangana. ""Ston...",1
1,1,ablaze,NaN,Telangana: Section 144 has been imposed in Bha...,1
2,2,ablaze,New York City,Arsonist sets cars ablaze at dealership https:...,1
3,3,ablaze,"Morgantown, WV",Arsonist sets cars ablaze at dealership https:...,1
4,4,ablaze,NaN,"""Lord Jesus, your love brings freedom and pard...",0
...,...,...,...,...,...
11365,11365,wrecked,Blue State in a red sea,Media should have warned us well in advance. T...,0
11366,11366,wrecked,arohaonces,i feel directly attacked 💀 i consider moonbin ...,0
11367,11367,wrecked,🇵🇭,i feel directly attacked 💀 i consider moonbin ...,0
11368,11368,wrecked,auroraborealis,"ok who remember ""outcast"" nd the ""dora"" au?? T...",0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11370 entries, 0 to 11369
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        11370 non-null  int64 
 1   keyword   11370 non-null  object
 2   location  7952 non-null   object
 3   text      11370 non-null  object
 4   target    11370 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 444.3+ KB


In [ ]:
df.isnull().sum()

id             0
keyword        0
location    3418
text           0
target         0
dtype: int64

In [ ]:
df['target'].value_counts()

0    9256
1    2114
Name: target, dtype: int64

In [ ]:
df['location'].value_counts()

United States              96
Australia                  83
London, England            81
UK                         77
India                      74
                           ..
Great State of Texas        1
Karatina, Kenya             1
The internet or the gym     1
Reston, VA                  1
auroraborealis              1
Name: location, Length: 4504, dtype: int64

In [ ]:
df.drop('id',inplace=True,axis=1)

In [ ]:
df.head()

,keyword,location,text,target
0,ablaze,NaN,"Communal violence in Bhainsa, Telangana. ""Ston...",1
1,ablaze,NaN,Telangana: Section 144 has been imposed in Bha...,1
2,ablaze,New York City,Arsonist sets cars ablaze at dealership https:...,1
3,ablaze,"Morgantown, WV",Arsonist sets cars ablaze at dealership https:...,1
4,ablaze,NaN,"""Lord Jesus, your love brings freedom and pard...",0


In [ ]:
df.drop(['keyword','location'],inplace=True,axis=1)

In [ ]:
df.head()

,text,target
0,"Communal violence in Bhainsa, Telangana. ""Ston...",1
1,Telangana: Section 144 has been imposed in Bha...,1
2,Arsonist sets cars ablaze at dealership https:...,1
3,Arsonist sets cars ablaze at dealership https:...,1
4,"""Lord Jesus, your love brings freedom and pard...",0


In [ ]:
df['text'].value_counts()

I want to help you with my project to save the Caribbean Sea from floods and hurricanes https://t.co/qD8Om9NqQK                              3
We wanted to entertain you all with a good movie on Sankranthi festival. And you gave us a Landslide Victory as return gift.…                3
"Rumours of a trial of no notice inspections". Have they completely lost the plot? https://t.co/EvCqSi8zI0                                   2
Gov. Wolf vows to veto bill loosening rules for conventional oil and gas wells https://t.co/T1qYcGo7TQ                                       2
Chemical Hazard - Advice for Cobram. For more info: https://t.co/L9uP0CsCJL #vichazmat                                                       2
                                                                                                                                            ..
It is glorious. Hiu provides a very great service in this desolate world of ours                                                             1

In [ ]:
pip install emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 353.7/353.7 kB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.4.0-py2.py3-none-any.whl size=350809 sha256=9229600c793c21d7ba14b2c0bdfd551e9856288e251536399b158c01191f9201
  Stored in directory: /root/.cache/pip/wheels/79/29/1c/234cae4632803c2ba4a76a71a679eb1383cf590775714e2a21
Successfully built emoji


In [ ]:
import regex as re
import emoji

In [ ]:
def cleantext(text):
  tokens = word_tokenize(text.lower())
  ftoken = [t for t in tokens if(t.isalpha())]

  stop = stopwords.words("english")
  ctoken = [t for t in ftoken if(t not in stop)]

  lemma = WordNetLemmatizer()
  ltoken = [lemma.lemmatize(t) for t in ctoken]
  
  return " ".join(ltoken)

In [ ]:
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U0001F921"
                           u"\U0001F92A"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
   
    return emoji_pattern.sub(r'', text)

In [ ]:
def preprocess(text):
    text=text.lower()
    text= re.sub("[\\n,\n,\t,\\,\\,$.,#,'\',]", "", text)
    text=re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ", text)
    text = re.sub("\[.*?\]","", text)#removes . * ? 
    text=text.strip()# removes trailing spaces from left and right
    text=re.sub(emoji.emojize("\N{clown face}"),"",text)
    text=re.sub(emoji.emojize("\N{Face with monocle}"),"",text)
    text=re.sub(emoji.emojize("\r{red heart}"),"",text)
    text=re.sub(emoji.emojize(u"\U0001F92A"),"",text)#🤪
    text=re.sub(emoji.emojize(u"\U0001F917"),"",text)#🤗
    text=re.sub(emoji.emojize(u"\U0001F914"),"",text)#🤔
    text=re.sub(emoji.emojize(u"\U0001F97A"),"",text)#🥺
    text=re.sub(emoji.emojize(u"\U0001F937"),"",text)#🤷
    text=re.sub(emoji.emojize(u"\U0001F973"),"",text)#🥳
    text=re.sub(emoji.emojize(u"\U0001F923"),"",text)#🤣
    text=re.sub(emoji.emojize(u"\U0001F913"),"",text)#🤓
    text=re.sub(emoji.emojize(u"\U0001F926"),"",text)#🤦
#     text=re.sub(emoji.emojize(u"\U00023EA"),"",text)#⏪ this doesn't work
    text=remove_emoji(text)
    return text

In [ ]:
df['clean_text']=df['text'].apply(preprocess)

In [ ]:
df['clean_text']=df['text'].apply(cleantext)

In [ ]:
pip install emot

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 2.4 MB/s eta 0:00:00


In [ ]:
import emot
emot_obj=emot.core.emot()

In [ ]:
emot_obj.bulk_emoji(df['text'])

Issue with internal pattern finding emoji: '🇻'


[{'value': [], 'location': [], 'mean': [], 'flag': False},
 {'value': [], 'location': [], 'mean': [], 'flag': False},
 {'value': [], 'location': [], 'mean': [], 'flag': False},
 {'value': [], 'location': [], 'mean': [], 'flag': False},
 {'value': [], 'location': [], 'mean': [], 'flag': False},
 {'value': [], 'location': [], 'mean': [], 'flag': False},
 {'value': [], 'location': [], 'mean': [], 'flag': False},
 {'value': [], 'location': [], 'mean': [], 'flag': False},
 {'value': [], 'location': [], 'mean': [], 'flag': False},
 {'value': [], 'location': [], 'mean': [], 'flag': False},
 {'value': [], 'location': [], 'mean': [], 'flag': False},
 {'value': [], 'location': [], 'mean': [], 'flag': False},
 {'value': [], 'location': [], 'mean': [], 'flag': False},
 {'value': [], 'location': [], 'mean': [], 'flag': False},
 {'value': [], 'location': [], 'mean': [], 'flag': False},
 {'value': [], 'location': [], 'mean': [], 'flag': False},
 {'value': [], 'location': [], 'mean': [], 'flag': False

In [ ]:
emot_obj.bulk_emoji(df['clean_text'])

[{'value': [], 'location': [], 'mean': [], 'flag': False},
 {'value': [], 'location': [], 'mean': [], 'flag': False},
 {'value': [], 'location': [], 'mean': [], 'flag': False},
 {'value': [], 'location': [], 'mean': [], 'flag': False},
 {'value': [], 'location': [], 'mean': [], 'flag': False},
 {'value': [], 'location': [], 'mean': [], 'flag': False},
 {'value': [], 'location': [], 'mean': [], 'flag': False},
 {'value': [], 'location': [], 'mean': [], 'flag': False},
 {'value': [], 'location': [], 'mean': [], 'flag': False},
 {'value': [], 'location': [], 'mean': [], 'flag': False},
 {'value': [], 'location': [], 'mean': [], 'flag': False},
 {'value': [], 'location': [], 'mean': [], 'flag': False},
 {'value': [], 'location': [], 'mean': [], 'flag': False},
 {'value': [], 'location': [], 'mean': [], 'flag': False},
 {'value': [], 'location': [], 'mean': [], 'flag': False},
 {'value': [], 'location': [], 'mean': [], 'flag': False},
 {'value': [], 'location': [], 'mean': [], 'flag': False

In [ ]:
df.head()

,text,target,clean_text
0,"Communal violence in Bhainsa, Telangana. ""Ston...",1,communal violence bhainsa telangana stone pelt...
1,Telangana: Section 144 has been imposed in Bha...,1,telangana section imposed bhainsa january clas...
2,Arsonist sets cars ablaze at dealership https:...,1,arsonist set car ablaze dealership http
3,Arsonist sets cars ablaze at dealership https:...,1,arsonist set car ablaze dealership http http
4,"""Lord Jesus, your love brings freedom and pard...",0,lord jesus love brings freedom pardon fill hol...


In [ ]:
x = df["clean_text"]
y = df["target"]

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.3, random_state=1)

In [ ]:
sentlen = []

for sent in df["clean_text"]:
  sentlen.append(len(word_tokenize(sent)))     #sentlen.append(len(i.split(sent)))

df["SentLen"] = sentlen 
df.head()

,text,target,clean_text,SentLen
0,"Communal violence in Bhainsa, Telangana. ""Ston...",1,communal violence bhainsa telangana stone pelt...,11
1,Telangana: Section 144 has been imposed in Bha...,1,telangana section imposed bhainsa january clas...,10
2,Arsonist sets cars ablaze at dealership https:...,1,arsonist set car ablaze dealership http,6
3,Arsonist sets cars ablaze at dealership https:...,1,arsonist set car ablaze dealership http http,7
4,"""Lord Jesus, your love brings freedom and pard...",0,lord jesus love brings freedom pardon fill hol...,13


In [ ]:
max(sentlen)

23

In [ ]:
np.quantile(sentlen, 0.95)

14.0

In [ ]:
max_len = np.quantile(sentlen, 0.95)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
tok = Tokenizer(char_level=False, split=" ")
#char_level	if True, every character will be treated as a token.

tok.fit_on_texts(xtrain)
tok.index_word

{1: 'http',
 2: 'amp',
 3: 'people',
 4: 'like',
 5: 'fire',
 6: 'one',
 7: 'u',
 8: 'get',
 9: 'year',
 10: 'need',
 11: 'time',
 12: 'know',
 13: 'new',
 14: 'would',
 15: 'day',
 16: 'iran',
 17: 'say',
 18: 'see',
 19: 'first',
 20: 'storm',
 21: 'want',
 22: 'go',
 23: 'emergency',
 24: 'still',
 25: 'think',
 26: 'death',
 27: 'let',
 28: 'today',
 29: 'two',
 30: 'trump',
 31: 'man',
 32: 'world',
 33: 'back',
 34: 'could',
 35: 'body',
 36: 'make',
 37: 'australia',
 38: 'got',
 39: 'many',
 40: 'please',
 41: 'news',
 42: 'really',
 43: 'home',
 44: 'woman',
 45: 'right',
 46: 'also',
 47: 'attack',
 48: 'bomb',
 49: 'take',
 50: 'last',
 51: 'help',
 52: 'much',
 53: 'said',
 54: 'nuclear',
 55: 'never',
 56: 'look',
 57: 'killed',
 58: 'volcano',
 59: 'good',
 60: 'love',
 61: 'service',
 62: 'come',
 63: 'going',
 64: 'life',
 65: 'taal',
 66: 'building',
 67: 'another',
 68: 'way',
 69: 'thing',
 70: 'suicide',
 71: 'show',
 72: 'even',
 73: 'train',
 74: 'mass',
 75: 'sta

In [ ]:
vocab_len = len(tok.index_word)
vocab_len

13957

In [ ]:
seqtrain = tok.texts_to_sequences(xtrain) #step1
seqtrain

[[2706,
  1503,
  4341,
  131,
  314,
  2012,
  2013,
  4342,
  949,
  97,
  4343,
  13,
  796,
  2323,
  1],
 [2707, 674, 2707, 898, 634, 1000, 2708, 1640, 797, 128, 29, 510, 180],
 [81, 4, 851, 104, 14, 123, 488, 4344, 397, 798],
 [252, 421, 329, 398, 1352, 2324, 6, 530, 421, 1, 3335],
 [105, 1001, 1173, 330, 1001, 6542, 1],
 [1641, 2325, 1174, 4, 552, 6543, 4345, 2709, 4346, 2326, 2014, 511, 635, 1],
 [4347, 1642, 1803, 27, 39, 1353, 2710, 49, 512, 1002, 89],
 [753, 6544, 3336, 2711, 4348, 2712, 78, 344, 852, 1175, 613, 33, 950],
 [2015, 4349, 6545, 3337, 6546, 2713, 951, 2327, 4349, 1],
 [636, 358, 1176, 511, 2, 4350, 4351],
 [1804, 4352, 2714, 635, 3338, 553, 2016, 134],
 [4353, 754, 489, 488, 853, 531, 709, 1088, 72, 66, 1],
 [224, 263, 55, 6547, 242, 55, 2328, 264, 675, 6548],
 [302, 6549, 4354, 359, 6550, 225, 4355, 1],
 [1003, 637, 28, 106, 234, 2017, 1],
 [1089, 582, 1251, 1354, 583, 463, 218, 43, 57, 39, 1],
 [235, 2018, 440, 4356, 1],
 [710, 44, 711, 676, 360, 799, 1, 1],
 

In [ ]:
from tensorflow.keras.preprocessing import sequence

In [ ]:
seqmattrain = sequence.pad_sequences(seqtrain, maxlen= int(max_len)) #step2
seqmattrain

array([[1503, 4341,  131, ...,  796, 2323,    1],
       [   0, 2707,  674, ...,   29,  510,  180],
       [   0,    0,    0, ..., 4344,  397,  798],
       ...,
       [   0,    0,    0, ...,   25, 1687, 1725],
       [   0,    0,    0, ..., 2277,  562,    1],
       [   0,    0,    0, ...,    4,  327,  551]], dtype=int32)

In [ ]:
seqtest = tok.texts_to_sequences(xtest)
seqmattest = sequence.pad_sequences(seqtest, maxlen=int(max_len))

In [ ]:
vocab_len


13957

In [ ]:
rnn = Sequential()

rnn.add(Embedding(vocab_len+1,700, input_length=int(max_len), mask_zero=True))
rnn.add(SimpleRNN(units=32, activation="tanh"))
rnn.add(Dense(units=32, activation="relu"))
rnn.add(Dropout(0.2))

rnn.add(Dense(units=1, activation="sigmoid"))

rnn.compile(optimizer="adam", loss="binary_crossentropy")

rnn.fit(seqmattrain, ytrain, batch_size=25, epochs=25)

ypred = rnn.predict(seqmattest)
ypred


Epoch 1/25
319/319 [==============================] - 75s 230ms/step - loss: 0.3754
Epoch 2/25
319/319 [==============================] - 73s 230ms/step - loss: 0.0916
Epoch 3/25
319/319 [==============================] - 76s 237ms/step - loss: 0.0232
Epoch 4/25
319/319 [==============================] - 73s 229ms/step - loss: 0.0105
Epoch 5/25
319/319 [==============================] - 73s 230ms/step - loss: 0.0054
Epoch 6/25
319/319 [==============================] - 75s 236ms/step - loss: 0.0033
Epoch 7/25
319/319 [==============================] - 69s 217ms/step - loss: 0.0027
Epoch 8/25
319/319 [==============================] - 68s 214ms/step - loss: 0.0027
Epoch 9/25
319/319 [==============================] - 69s 216ms/step - loss: 0.0025
Epoch 10/25
319/319 [==============================] - 68s 212ms/step - loss: 0.0023
Epoch 11/25
319/319 [==============================] - 68s 213ms/step - loss: 0.0024
Epoch 12/25
319/319 [==============================] - 69s 217ms/step - lo

array([[2.1078238e-01],
       [4.6745587e-02],
       [3.3521679e-05],
       ...,
       [2.2268177e-03],
       [4.8560192e-08],
       [9.9967867e-01]], dtype=float32)

In [ ]:
ypred = ypred>0.5
ypred = np.where(ypred<0.5,0,1)
print(classification_report(ytest,ypred))

              precision    recall  f1-score   support

           0       0.91      0.91      0.91      2788
           1       0.59      0.59      0.59       623

    accuracy                           0.85      3411
   macro avg       0.75      0.75      0.75      3411
weighted avg       0.85      0.85      0.85      3411

